Design Rates
Palled Unload = 15 Pallets/hour
Fluid Unload = 840 Packages/hour
National Feeder = 22 Pallets/hour
National Inductor = 800 packages/hour
National Pallet = 209 Packages/hour
National Fluid = 273 packages/hour
Poly_Sort_NC = 160 Packages/hour
Non_con_NC = 36 Packages/hour
Pallet Load_NC = 9 Pallets/hour
TLMD Partition = 148 packages/hour
tlmd feeder = 15 pallets/hour
tlmd inductor = 300 packages/hour
tlmd sort = 84 packages/hour
tlmd cart handoff = 22 carts/hour


Assumed ratios
40 NC Box Packages = 1 Pallet
500 NC Poly Packages = 1 Pallet
15 NC Non-Con Packages = 1 Pallet
60 TLMD Boxes = 1 Pallet
150 TLMD Poly = 1 Pallet
5 TLMD Non-con Packages = 1 Pallet
T20 TLMD packages = 1 cart
55 Inbound Packages = 1 Inbound Pallet 

In [3]:
from scipy.optimize import linprog

inbound_pallet_to_package_conversion = 55 #Packages/pallet
NC_pallet_to_package_conversion = 40 #Packages/pallet
tlmd_pallet_to_package_conversion = 60
tlmd_cart_to_packages_conversion = 20


#all rates converted to packages/hour
pallet_unload = 15 * inbound_pallet_to_package_conversion
fluid_unload = 840
national_feeder = 22 * inbound_pallet_to_package_conversion 
national_inductor = 800
national_pallet_box = 209
national_fluid = 273
Poly_Sort_NC = 160
Non_con_NC = 36
pallet_Load_NC = 9 * NC_pallet_to_package_conversion
tLMD_partition = 148
tlmd_feeder = 15 * tlmd_pallet_to_package_conversion
tlmd_inductor = 300
tlmd_sort = 84
tlmd_cart_handoff = 22 * tlmd_cart_to_packages_conversion

salary_rate = 21.5

line_haul_packages = 9081
TFC_packages = 1821
tlmd_packages = 7280
NC_packages = 3622
NC_box = NC_packages*.743
NC_poly = NC_packages*.214
NC_non_con = NC_packages*.043


# Define the hourly rates for each station (example values)
hourly_rates = [salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate, 
                salary_rate,
                salary_rate,
                salary_rate
                ]

# Define the packages per hour rate for each station (example values)
packages_per_hour = [pallet_unload, 
                    fluid_unload, 
                    national_feeder, 
                    national_inductor, 
                    national_pallet_box, 
                    national_fluid, 
                    Poly_Sort_NC, 
                    Non_con_NC, 
                    pallet_Load_NC, 
                    tLMD_partition, 
                    tlmd_feeder, 
                    tlmd_inductor,
                    tlmd_sort,
                    tlmd_cart_handoff
                    ]

# Define the number of packages to be processed through each station (example values)
packages_processed = [line_haul_packages, # number of packages non-TFC
                    TFC_packages,  # number of packages TFC
                    line_haul_packages, # number of packages non-TFC
                    line_haul_packages + TFC_packages, # number of packages total
                    NC_box, #number of NC boxes
                    NC_box, #number of NC boxes
                    NC_poly, #number of NC poly
                    NC_non_con, #number of NC non-con
                    NC_box+NC_poly+NC_non_con, #number of total NC packages
                    tlmd_packages, #number of total TLMD packages
                    tlmd_packages, #number of total TLMD packages
                    tlmd_packages,#number of total TLMD packages
                    tlmd_packages, #number of total TLMD packages
                    tlmd_packages #number of total TLMD packages
                    ]

# Number of stations
num_stations = len(hourly_rates)

# Objective function coefficients (hourly rates)
c = hourly_rates

# Inequality constraints matrix (number of hours at a station times the packages per hour rate)
A = [[-packages_per_hour[i] if i == j else 0 for j in range(num_stations)] for i in range(num_stations)]

# Inequality constraints vector (negative number of packages processed through those stations)
b = [-packages_processed[i] for i in range(num_stations)]

# Bounds for each variable (number of hours at each station must be non-negative)
x_bounds = [(0, None) for _ in range(num_stations)]

# Solve the linear programming problem
result = linprog(c=c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Print the results
if result.success:
    print(f"Optimal number of hours at each station: {result.x}")
    print(f"Minimum cost: {result.fun}")
else:
    print("Optimization failed.")

Optimal number of hours at each station: [11.00727273  2.16785714  7.50495868 13.6275     12.87629665  9.85767766
  4.844425    4.32627778 10.06111111 49.18918919  8.08888889 24.26666667
 86.66666667 16.54545455]
Minimum cost: 5612.150218049269


In [1]:
import pulp

# Define the conversion rates and packages per hour
inbound_pallet_to_package_conversion = 55
NC_pallet_to_package_conversion = 40
tlmd_pallet_to_package_conversion = 60
tlmd_cart_to_packages_conversion = 20

# All rates converted to packages/hour
pallet_unload = 15 * inbound_pallet_to_package_conversion
fluid_unload = 840
national_feeder = 22 * inbound_pallet_to_package_conversion 
national_inductor = 800
national_pallet_box = 209
national_fluid = 273
Poly_Sort_NC = 160
Non_con_NC = 36
pallet_Load_NC = 9 * NC_pallet_to_package_conversion
tLMD_partition = 148
tlmd_feeder = 15 * tlmd_pallet_to_package_conversion
tlmd_inductor = 300
tlmd_sort = 84
tlmd_cart_handoff = 22 * tlmd_cart_to_packages_conversion

salary_rate = 21.5

line_haul_packages = 9081
TFC_packages = 1821
tlmd_packages = 7280
NC_packages = 3622
NC_box = NC_packages * 0.743
NC_poly = NC_packages * 0.214
NC_non_con = NC_packages * 0.043

# Define the hourly rates for each station
hourly_rates = [salary_rate] *14

# Define the packages per hour rate for each station
packages_per_hour = [pallet_unload, fluid_unload, national_feeder, national_inductor, national_pallet_box, national_fluid, 
                     Poly_Sort_NC, Non_con_NC, pallet_Load_NC, tLMD_partition, tlmd_feeder, tlmd_inductor, tlmd_sort, tlmd_cart_handoff]

# Define the number of packages to be processed through each station
packages_processed = [line_haul_packages, TFC_packages, line_haul_packages, line_haul_packages + TFC_packages, NC_box, NC_box, 
                      NC_poly, NC_non_con, NC_box + NC_poly + NC_non_con, tlmd_packages, tlmd_packages, tlmd_packages, tlmd_packages, tlmd_packages]

# Create a problem variable
prob = pulp.LpProblem("StaffingProblem", pulp.LpMinimize)

# Decision variables for hours at each station
hours = [pulp.LpVariable(f'hours_{i}', lowBound=0, cat='Continuous') for i in range(len(hourly_rates))]

# Binary decision variables for staffing options
x = pulp.LpVariable('x', cat='Binary')  # national_fluid
y = pulp.LpVariable('y', cat='Binary')  # national_pallet_box

# Objective function (minimize total salary cost)
prob += pulp.lpSum([hourly_rates[i] * hours[i] for i in range(len(hourly_rates))])

# Constraints for packages processed
for i in range(len(packages_per_hour)):
    prob += hours[i] * packages_per_hour[i] >= packages_processed[i]

# Binary decision constraints
prob += x + y == 1, "OneStaffingOption"
prob += hours[5] >= 1 * x, "MinStaffFluid"  # national_fluid
prob += hours[4] >= 1 * y, "MinStaffPalletBox"  # national_pallet_box

# Solve the problem
prob.solve()

# Print the results
print(f"Status: {pulp.LpStatus[prob.status]}")
for i in range(len(hours)):
    print(f"Optimal number of hours at station {i}: {hours[i].varValue}")
print(f"Minimum cost: {pulp.value(prob.objective)}")
print(f"Decision variable for national_pallet_box (y): {y.varValue}")
print(f"Decision variable for national_fluid (x): {x.varValue}")


Status: Optimal
Optimal number of hours at station 0: 11.007273
Optimal number of hours at station 1: 2.1678571
Optimal number of hours at station 2: 7.5049587
Optimal number of hours at station 3: 13.6275
Optimal number of hours at station 4: 12.876297
Optimal number of hours at station 5: 9.8576777
Optimal number of hours at station 6: 4.844425
Optimal number of hours at station 7: 4.3262778
Optimal number of hours at station 8: 10.061111
Optimal number of hours at station 9: 49.189189
Optimal number of hours at station 10: 8.0888889
Optimal number of hours at station 11: 24.266667
Optimal number of hours at station 12: 86.666667
Optimal number of hours at station 13: 16.545455
Minimum cost: 5612.150250299999
Decision variable for national_pallet_box (y): 1.0
Decision variable for national_fluid (x): 0.0


In [3]:
from pyscipopt import Model, quicksum

# Define the conversion rates and packages per hour
inbound_pallet_to_package_conversion = 55
NC_pallet_to_package_conversion = 40
tlmd_pallet_to_package_conversion = 60
tlmd_cart_to_packages_conversion = 20

# All rates converted to packages/hour
pallet_unload = 15 * inbound_pallet_to_package_conversion
fluid_unload = 840
national_feeder = 22 * inbound_pallet_to_package_conversion 
national_inductor = 800
national_pallet_box = 209
national_fluid = 273
Poly_Sort_NC = 160
Non_con_NC = 36
pallet_Load_NC = 9 * NC_pallet_to_package_conversion
tLMD_partition = 148
tlmd_feeder = 15 * tlmd_pallet_to_package_conversion
tlmd_inductor = 300
tlmd_sort = 84
tlmd_cart_handoff = 22 * tlmd_cart_to_packages_conversion

salary_rate = 21.5

line_haul_packages = 9081
TFC_packages = 1821
tlmd_packages = 7280
NC_packages = 3622
NC_box = NC_packages * 0.743
NC_poly = NC_packages * 0.214
NC_non_con = NC_packages * 0.043

# Define the hourly rates for each station
hourly_rates = [salary_rate] * 14

# Define the packages per hour rate for each station
packages_per_hour = [pallet_unload, fluid_unload, national_feeder, national_inductor, national_pallet_box, national_fluid, 
                     Poly_Sort_NC, Non_con_NC, pallet_Load_NC, tLMD_partition, tlmd_feeder, tlmd_inductor, tlmd_sort, tlmd_cart_handoff]

# Define the number of packages to be processed through each station
packages_processed = [line_haul_packages, TFC_packages, line_haul_packages, line_haul_packages + TFC_packages, NC_box, NC_box, 
                      NC_poly, NC_non_con, NC_box + NC_poly + NC_non_con, tlmd_packages, tlmd_packages, tlmd_packages, tlmd_packages, tlmd_packages]

# Create a SCIP model
model = Model("StaffingProblem")

# Decision variables for hours at each station
hours = [model.addVar(f'hours_{i}', lb=0) for i in range(len(hourly_rates))]

# Binary decision variables for staffing options
x = model.addVar('x', vtype='B')  # national_fluid
y = model.addVar('y', vtype='B')  # national_pallet_box

# Objective function (minimize total salary cost)
model.setObjective(quicksum(hourly_rates[i] * hours[i] for i in range(len(hourly_rates))), "minimize")

# Constraints for packages processed
for i in range(len(packages_per_hour)):
    model.addCons(hours[i] * packages_per_hour[i] >= packages_processed[i])

# Binary decision constraints
model.addCons(x + y == 1, "OneStaffingOption")
model.addCons(hours[5] >= 1 * x, "MinStaffFluid")  # national_fluid
model.addCons(hours[4] >= 1 * y, "MinStaffPalletBox")  # national_pallet_box

# Solve the problem
model.optimize()

# Print the results
if model.getStatus() == "optimal":
    print(f"Status: {model.getStatus()}")
    for i in range(len(hours)):
        print(f"Optimal number of hours at station {i}: {model.getVal(hours[i])}")
    print(f"Minimum cost: {model.getObjVal()}")
    print(f"Decision variable for national_pallet_box (y): {model.getVal(y)}")
    print(f"Decision variable for national_fluid (x): {model.getVal(x)}")
else:
    print(f"Problem could not be solved to optimality. Status: {model.getStatus()}")


Status: optimal
Optimal number of hours at station 0: 11.007272727272728
Optimal number of hours at station 1: 2.167857142857143
Optimal number of hours at station 2: 7.50495867768595
Optimal number of hours at station 3: 13.6275
Optimal number of hours at station 4: 12.876296650717704
Optimal number of hours at station 5: 9.857677655677657
Optimal number of hours at station 6: 4.844425
Optimal number of hours at station 7: 4.3262777777777774
Optimal number of hours at station 8: 10.061111111111112
Optimal number of hours at station 9: 49.18918918918919
Optimal number of hours at station 10: 8.088888888888889
Optimal number of hours at station 11: 24.26666666666667
Optimal number of hours at station 12: 86.66666666666666
Optimal number of hours at station 13: 16.545454545454543
Minimum cost: 5612.150218049271
Decision variable for national_pallet_box (y): -0.0
Decision variable for national_fluid (x): 1.0


Induct/NC Optimization

In [8]:
from pyscipopt import Model, quicksum
import pyscipopt as scip
from pyscipopt.recipe.nonlinear import set_nonlinear_objective

# Define the conversion rates and packages per hour
inbound_pallet_to_package_conversion = 55
NC_pallet_to_package_conversion = 40
tlmd_pallet_to_package_conversion = 60
tlmd_cart_to_packages_conversion = 20

# All rates converted to packages/hour
pallet_unload = 15 * inbound_pallet_to_package_conversion
fluid_unload = 840
national_feeder = 22 * inbound_pallet_to_package_conversion 
national_inductor = 800
national_pallet_box = 209
USPS_fluid = 273
UPSN_fluid = 273
FDEG_fluid = 273
Poly_Sort_NC = 160
Non_con_NC = 36
pallet_Load_NC = 9 * NC_pallet_to_package_conversion
tlmd_partition = 148



salary_rate = 21.5


total_TMs = 50
line_haul_packages = 9081
TFC_packages = 1821
tlmd_packages = 7280
NC_packages = 3622

NC_box = NC_packages * 0.743
NC_poly = NC_packages * 0.214
NC_non_con = NC_packages * 0.043

NC_box_USPS = NC_box * 0.5
NC_box_UPSN = NC_box * 0.3
NC_box_FDEG = NC_box * 0.2

# Define the hourly rates for each station
hourly_rates = [salary_rate] * 12


# Create a SCIP model
model = Model("StaffingProblem")

# Binary decision variables for fluid options
x = model.addVar('x', vtype='B')  # national_fluid_USPS
y = model.addVar('y', vtype='B')  # national_fluid_UPSN
z = model.addVar('z', vtype='B')  # national_fluid_FDEG

# Define the packages per hour rate for each station
packages_per_hour = [pallet_unload, fluid_unload, national_feeder, national_inductor, national_pallet_box, USPS_fluid, UPSN_fluid, FDEG_fluid, 
                     Poly_Sort_NC, Non_con_NC, pallet_Load_NC, tlmd_partition]

# Define the number of packages to be processed through each station
packages_processed = [line_haul_packages, TFC_packages, line_haul_packages, line_haul_packages + TFC_packages, NC_box - x*NC_box_USPS - y*NC_box_UPSN - z*NC_box_FDEG, NC_box_USPS, NC_box_UPSN, NC_box_FDEG, 
                      NC_poly, NC_non_con, NC_box + NC_poly + NC_non_con, tlmd_packages]



big_M = 1000
# Decision variables for hours at each station
hours = [model.addVar(f'hours_{i}', lb=0) for i in range(len(hourly_rates))]
TMs = [model.addVar(f'TMs_{i}', vtype='I', lb=0) for i in range(len(hourly_rates))]

# Objective function (minimize total salary cost)
set_nonlinear_objective(model, scip.quicksum(TMs[i] * hourly_rates[i] * hours[i] for i in range(len(hourly_rates))), "minimize")

# Constraints for packages processed
for i in range(len(packages_per_hour)):
    model.addCons(hours[i] * packages_per_hour[i] >= packages_processed[i])

#constraint for total number of team members used
model.addCons(quicksum(TMs[i] for i in range(len(hourly_rates))) == total_TMs)

# Constraint to ensure that fluid load is staffed correctly based on the binary decision variables
model.addCons(TMs[6] >= 3 - big_M * (1 - x))
model.addCons(TMs[6] <= big_M * x)

model.addCons(TMs[7] >= 3 - big_M * (1 - y))
model.addCons(TMs[7] <= big_M * y)

model.addCons(TMs[8] >= 3 - big_M * (1 - z))
model.addCons(TMs[8] <= big_M * z)

# Solve the problem
model.optimize()

# Print the results
if model.getStatus() == "optimal":
    print(f"Status: {model.getStatus()}")
    for i in range(len(hours)):
        print(f"Optimal number of hours at station {i}: {model.getVal(hours[i])}")
    print(f"Minimum cost: {model.getObjVal()}")
    print(f"Decision variable for national_pallet_box (y): {model.getVal(y)}")
    print(f"Decision variable for national_fluid (x): {model.getVal(x)}")
else:
    print(f"Problem could not be solved to optimality. Status: {model.getStatus()}")


ModuleNotFoundError: No module named 'pyscipopt.recipe'

TLMD Optimization

In [ ]:
from pyscipopt import Model, quicksum

# Define the conversion rates and packages per hour
inbound_pallet_to_package_conversion = 55
NC_pallet_to_package_conversion = 40
tlmd_pallet_to_package_conversion = 60
tlmd_cart_to_packages_conversion = 20

# All rates converted to packages/hour
pallet_unload = 15 * inbound_pallet_to_package_conversion
fluid_unload = 840
national_feeder = 22 * inbound_pallet_to_package_conversion 
national_inductor = 800
national_pallet_box = 209
national_fluid = 273
Poly_Sort_NC = 160
Non_con_NC = 36
pallet_Load_NC = 9 * NC_pallet_to_package_conversion
tLMD_partition = 148
tlmd_feeder = 15 * tlmd_pallet_to_package_conversion
tlmd_inductor = 300
tlmd_sort = 84
tlmd_cart_handoff = 22 * tlmd_cart_to_packages_conversion

salary_rate = 21.5

line_haul_packages = 9081
TFC_packages = 1821
tlmd_packages = 7280
NC_packages = 3622
NC_box = NC_packages * 0.743
NC_poly = NC_packages * 0.214
NC_non_con = NC_packages * 0.043

# Define the hourly rates for each station
hourly_rates = [salary_rate] * 14

# Define the packages per hour rate for each station
packages_per_hour = [pallet_unload, fluid_unload, national_feeder, national_inductor, national_pallet_box, national_fluid, 
                     Poly_Sort_NC, Non_con_NC, pallet_Load_NC, tLMD_partition, tlmd_feeder, tlmd_inductor, tlmd_sort, tlmd_cart_handoff]

# Define the number of packages to be processed through each station
packages_processed = [line_haul_packages, TFC_packages, line_haul_packages, line_haul_packages + TFC_packages, NC_box, NC_box, 
                      NC_poly, NC_non_con, NC_box + NC_poly + NC_non_con, tlmd_packages, tlmd_packages, tlmd_packages, tlmd_packages, tlmd_packages]

# Create a SCIP model
model = Model("StaffingProblem")

# Decision variables for hours at each station
hours = [model.addVar(f'hours_{i}', lb=0) for i in range(len(hourly_rates))]

# Binary decision variables for staffing options
x = model.addVar('x', vtype='B')  # national_fluid
y = model.addVar('y', vtype='B')  # national_pallet_box

# Objective function (minimize total salary cost)
model.setObjective(quicksum(hourly_rates[i] * hours[i] for i in range(len(hourly_rates))), "minimize")

# Constraints for packages processed
for i in range(len(packages_per_hour)):
    model.addCons(hours[i] * packages_per_hour[i] >= packages_processed[i])

# Binary decision constraints
model.addCons(x + y == 1, "OneStaffingOption")
model.addCons(hours[5] >= 1 * x, "MinStaffFluid")  # national_fluid
model.addCons(hours[4] >= 1 * y, "MinStaffPalletBox")  # national_pallet_box

# Solve the problem
model.optimize()

# Print the results
if model.getStatus() == "optimal":
    print(f"Status: {model.getStatus()}")
    for i in range(len(hours)):
        print(f"Optimal number of hours at station {i}: {model.getVal(hours[i])}")
    print(f"Minimum cost: {model.getObjVal()}")
    print(f"Decision variable for national_pallet_box (y): {model.getVal(y)}")
    print(f"Decision variable for national_fluid (x): {model.getVal(x)}")
else:
    print(f"Problem could not be solved to optimality. Status: {model.getStatus()}")
